In [1]:
# adds parent dir to python path
import sys
sys.path.insert(0, '..')

#system lib
import os

# 3rd party lib
import numpy as np
#import matplotlib
#import matplotlib.pyplot as plt
#%matplotlib notebook
#matplotlib.rcParams.update({'font.size': 22})
#from bqplot import pyplot as plt

import matplotlib.pyplot as plt
%matplotlib auto

# astro lib
from astropy.io import fits
import sep

# my lib
import ImageTools as it
import DataTools as dt

from importlib import reload

import warnings
warnings.filterwarnings('ignore')

Using matplotlib backend: Qt5Agg


In [48]:
def make_nonnegative(img):
    epsilon = np.abs(img.min()) + 1e-3
    img[img<=0] = img[img>0].min()
    return img

def get_random_spheroids(num=10):
    with open('../spheroids', 'r') as f:
        spheroids = np.array(f.readlines())
    
    if num==-1:
        num = len(spheroids)
    
    selected = np.random.choice(spheroids, num, replace=False)
    
    data_dir = os.path.join(os.getenv('HOME'), 'Documents/astro_data/orig_images')
    fmask = 'GDS_{}_{}.fits'
    f_string = os.path.join(data_dir, fmask)

    sources = []
    for s in selected:
        img = fits.getdata(f_string.format(s.strip(), 'h'))
        segmap = fits.getdata(f_string.format(s.strip(), 'segmap'))
        img_id = int(s.split('_')[1])
        sources.append((img.copy(), segmap.copy(), img_id))
        del img 
        del segmap
    
    return sources

# Petrosian Helpers ----------------------------------------------------------
def one_over_eta(rs, fs, R):
    return ((np.pi*R**2) / (np.sum(fs[rs<=R]))) * fs[rs==R]

def petrosian_radius(rs, fs):
    #r_candidates = np.array([one_over_eta(rs, fs, R) for R in rs])
    r_candidates = np.array([fs[rs==r]/fs[rs<=r].mean() for r in rs]).flatten()
    r_candidates = r_candidates[rs<12.5]
    
    return rs[np.square(r_candidates-0.2).argmin()]

def petrosian_flux(rs, fs, R_p):
    return np.sum(fs[rs<=2*R_p])

def R_x(rs, fs, x):
    x /= 100
    sum_ratio = np.cumsum(fs)/np.sum(fs)
    return rs[np.square(sum_ratio-x).argmin()]

def petrosian_Re(R_50, R_90):
    P_3 = 8e-6
    P_4 = 8.47
    R_ratio = min(R_90/R_50, 3.8)
    return R_50 / (1 - (P_3 * (R_ratio)**P_4))

def get_rs_and_fs(img, src_map):
    cx, cy = it.img_center(img, src_map)
    xs, ys = np.meshgrid(np.arange(img.shape[0]), np.arange(img.shape[1]).T)
    rs = np.sqrt(np.square(xs-cx)+np.square(ys-cy))
    
    rs = rs.flatten()
    fs = img.flatten()
    sorted_rs = np.argsort(rs)
    
    rs = rs[sorted_rs]
    fs = fs[sorted_rs]
    
    return rs, fs

def get_re(img, src_map):
    rs, fs = get_rs_and_fs(img, src_map)
    
    R_p = petrosian_radius(rs, fs)
    F_p = petrosian_flux(rs, fs, R_p)
    p_mask = rs <= 2*R_p
    
    _rs = rs[p_mask]
    _fs = fs[p_mask]

    R_50 = R_x(_rs, _fs, 50)
    R_90 = R_x(_rs, _fs, 90)

    re = petrosian_Re(R_50, R_90)
    re_idx = np.square(_rs-re).argmin()
    
    return rs[re_idx], rs, fs

def get_re(rsfs):
    rs, fs = rsfs
    
    R_p = petrosian_radius(rs, fs)
    F_p = petrosian_flux(rs, fs, R_p)
    p_mask = rs <= 2*R_p
    
    _rs = rs[p_mask]
    _fs = fs[p_mask]

    R_50 = R_x(_rs, _fs, 50)
    R_90 = R_x(_rs, _fs, 90)

    re = petrosian_Re(R_50, R_90)
    re_idx = np.square(_rs-re).argmin()
    
    return rs[re_idx]

# Petrosian Helpers ----------------------------------------------------------

def denoise(img, segmap, img_id):
    noise_bank = img[segmap==0].flatten()
    other_source = np.logical_and(segmap!=0, segmap!=img_id)
    np.place(img, other_source, noise_bank)
    
    array_sep_likes = img.byteswap().newbyteorder()
    bkg = sep.Background(array_sep_likes, mask=segmap==img_id, bw=10, bh=10)
    img = img-bkg
    
    return img


In [50]:
reload(dt)

re_max_ratio = []
_spheroids = get_random_spheroids(num=-1)

res = []
ies = []
normed_rs = []
normed_fs = []
count = 0

small_res = []

for img, segmap, img_id in _spheroids:
    count += 1
    
    print(count/len(_spheroids), end='\r')

    #background subtract images
    src_map = segmap==img_id
    
    img = denoise(img, segmap, img_id)
    
    # get the sorted r vals and i vals
    rs, fs = get_rs_and_fs(img, src_map)
    
    # smooth the fs using loess
    #fs = dt.loessc(rs, fs, .60)
    fs = dt.loessc_p(rs, fs, 6.0, pnum=4)
    #print(len(rs), len(fs))
    #fs = lowess(fs, rs)
    #print(len(rs), len(fs))
    
    # measure petrosian re 
    re = get_re((rs,fs))

    
    if re<3:
        small_res.append((img_id, re))
        
    
    re_idx = np.where(rs==re)[0][0] 
    
    ie = fs[re_idx]
    
    #sum_to_re = fs[rs<=re_idx].sum()
    #src_L = sum_to_re*2
    #img_L = fs.sum()
    #if src_L>img_L:
    #    re_max_ratio.append(sum_to_re/img_L)
    #    continue
    
    #max_r = np.square(np.cumsum(fs)-src_L).argmin()
    
    #max_r = rs[max_r]
    
    #fs = fs
    #rs = rs
        
    normed_rs.append(rs)
    normed_fs.append(fs)
    ies.append(ie)
    res.append(re)    

In [52]:
plt.title('Effective Radius')
plt.xlabel('$R_e$(Pixels)')
plt.hist(res, bins=100)


(array([  1.,   0.,   0.,   0.,   1.,   0.,   0.,   1.,   2.,   4.,   4.,
          7.,   6.,  17.,  11.,  12.,  13.,  14.,  14.,  13.,  15.,  15.,
         15.,  16.,  13.,  15.,  13.,   9.,  12.,  12.,  19.,  10.,  17.,
         10.,  14.,  12.,   6.,   8.,   7.,   9.,   3.,  10.,   3.,   5.,
          4.,   3.,   2.,   3.,   5.,   4.,   6.,   2.,   3.,   5.,   3.,
          3.,   4.,   3.,   3.,   7.,   3.,   2.,   3.,   5.,   2.,   1.,
          1.,   1.,   0.,   3.,   1.,   1.,   1.,   2.,   0.,   1.,   2.,
          0.,   3.,   2.,   1.,   0.,   1.,   2.,   1.,   0.,   0.,   2.,
          1.,   2.,   2.,   1.,   1.,   1.,   2.,   0.,   0.,   0.,   0.,
          1.]),
 array([ 2.43620418,  2.48057556,  2.52494693,  2.56931831,  2.61368969,
         2.65806107,  2.70243245,  2.74680383,  2.7911752 ,  2.83554658,
         2.87991796,  2.92428934,  2.96866072,  3.0130321 ,  3.05740348,
         3.10177485,  3.14614623,  3.19051761,  3.23488899,  3.27926037,
         3.32363175,  3.36

In [30]:
plt.figure()
plt.title('Unnormalized Surface Brightness Profiles')
plt.xlabel('$R$')
plt.ylabel('$I(R)$')
for rs, fs, re, ie in zip(normed_rs, normed_fs, res, ies):
    cf = min(re/15.0, 1.0)
    color = (cf, 0.0, 1-cf, 0.2)
    plt.plot(rs, fs, color=color)
plt.show()

In [54]:
plt.figure()
plt.title('Normalized Surface Brightness Profiles')
plt.xlabel('$R/R_e$')
plt.ylabel('$I/I_e$')

for rs, fs, re, ie in zip(normed_rs, normed_fs, res, ies):
    cf = min(re/15.0, 1.0)
    color = (0.0, 0.0, 1.0, 0.2)
    if np.sum(fs<=0) > 0:
        fs = fs + np.abs(fs.min()) + 1e-3
        ie = fs[rs==re][0]
    
    plt.semilogy(rs/re, fs/ie, color=color)
plt.show()

In [60]:
interped_marks = np.linspace(0, 20, 20000)
stat_vals = []


plt.figure()
plt.title('Normalized Surface Brightness')
plt.xlabel('$R/R_e$')
plt.ylabel('$I/I_e$')

for rs, fs, re, ie in zip(normed_rs, normed_fs, res, ies):
    color = 'b'
    if np.sum(fs<=0) > 0:
        fs = fs + np.abs(fs.min()) + 1e-3
        ie = fs[rs==re][0]
    
    r = rs/re
    f = fs/ie
    plt.semilogy(r, f, color='k', alpha=.05)
    stat_vals.append(np.interp(interped_marks, r, f, left=np.nan, right=np.nan))
    

stat_vals = np.dstack(stat_vals)[0,...]
f_mean = np.nanmedian(stat_vals, axis=1)
f_16 = np.nanpercentile(stat_vals, 16, axis=1)
f_84 = np.nanpercentile(stat_vals, 84, axis=1)
    
plt.semilogy(interped_marks, f_mean, color='r', label='$mean$', zorder=100)
plt.fill_between(interped_marks, f_16, f_84, color='r', alpha=0.45, label="$16^{th}-84^{th}$", zorder=100)
plt.legend()
plt.show()

In [187]:
ratio_graph = lambda x: 8e-6 * x**8.47

plt.title('y=$8*10^{-6} * x^{8.47}$')
plt.semilogy(np.linspace(0, 10, 100), ratio_graph(np.linspace(0, 10, 100)))

In [195]:
#def petrosian_Re(R_50, R_90):
#    P_3 = 8e-6
#    P_4 = 8.47
#    return R_50 / (1 - P_3*(R_90/R_50)**P_4)
_re = []
_rp = []
for rs, fs in zip(normed_rs, normed_fs):
    P_3 = 8e-6
    P_4 = 8.47
    
    R_p = petrosian_radius(rs, fs)
    F_p = petrosian_flux(rs, fs, R_p)
    p_mask = rs <= 2*R_p
    
    _rs = rs[p_mask]
    _fs = fs[p_mask]

    R_50 = R_x(_rs, _fs, 50)
    R_90 = R_x(_rs, _fs, 90)

    re = petrosian_Re(R_50, R_90)
    re_idx = np.square(_rs-re).argmin()

    _re.append(rs[re_idx])
    _rp.append(R_p)
    if re < 3:
        print(f'{re} = {R_50} / (1 - {P_3} * ({R_90} / {R_50})**{P_4})')
      
    
    

2.9780318919337634 = 2.9339437555378596 / (1 - 8e-06 * (7.131842989867281 / 2.9339437555378596)**8.47)


In [196]:
plt.hist(_re, bins=100)

(array([  7.,  15.,  11.,  21.,  13.,  17.,  15.,  19.,  16.,   6.,   8.,
          3.,   3.,   5.,   7.,   4.,   7.,   3.,   3.,   5.,   5.,   7.,
          5.,   6.,   9.,   6.,  12.,   8.,  11.,  13.,  14.,  14.,  13.,
          7.,  19.,   9.,   9.,   9.,  10.,   2.,   8.,   3.,   6.,   2.,
          7.,   5.,   2.,   3.,   4.,   5.,   3.,   7.,   3.,   4.,   2.,
          2.,   3.,   3.,   4.,   2.,   1.,   2.,   2.,   2.,   3.,   3.,
          2.,   5.,   2.,   2.,   7.,   3.,   4.,   3.,   2.,   2.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          1.]), array([  3.0063464 ,   3.40447378,   3.80260117,   4.20072856,
          4.59885595,   4.99698334,   5.39511073,   5.79323812,
          6.19136551,   6.5894929 ,   6.98762029,   7.38574768,
          7.78387507,   8.18200246,   8.58012984,   8.97825723,
          9.37638462,   9.77451201,  10.1726394 ,  10.57076679,

In [111]:
plt.figure()
plt.title("SBP Comparing Small and Big Petrosian Radius Objects")
plt.xlabel('$R$')
#plt.ylabel('CumSum $I/I_e$ / Sum($I/I_e$)')
plt.ylabel('$I(R)$')

interped_marks = np.linspace(0.71, 5, 100)
stat_vals_small = []
stat_vals_big = []

log_scale=True
count = 0
for r, f, re, ie in zip(normed_rs, normed_fs, res, ies):
    
    if re<3:
        stat_vals_small.append(np.interp(interped_marks, r, f))
    else:
        stat_vals_big.append(np.interp(interped_marks, r, f))
    
    
    #selected = re < 3
    #if log_scale:
    #    if (f<=0).sum() > 0:
    #        f += np.abs(f.min()) + 1e-6

    #    if selected:
    #        stat_vals.append(np.interp(interped_marks, r, f))
    #        plt.semilogy(r,f, color='black', alpha=0.1)
    #else:
    #    if selected:
    #        stat_vals.append(np.interp(interped_marks, r, f))
    #        plt.plot(r, f, color='black', alpha=0.1)

stat_vals_big = np.dstack(stat_vals_big)[0,...]
big_mean = np.mean(stat_vals_big, axis=1)
big_16 = np.percentile(stat_vals_big, 16, axis=1)
big_84 = np.percentile(stat_vals_big, 84, axis=1)

stat_vals_small = np.dstack(stat_vals_small)[0,...]
small_mean = np.mean(stat_vals_small, axis=1)
small_16 = np.percentile(stat_vals_small, 16, axis=1)
small_84 = np.percentile(stat_vals_small, 84, axis=1)

plt.plot(interped_marks, small_mean, color='r', label='$mean(R_e < 3)$')
plt.fill_between(interped_marks, small_16, small_84, color='r', alpha=0.2, label="$16^{th}-84^{th}$")

plt.plot(interped_marks, big_mean, color='b', label='$mean(R_e >= 3)$')
plt.fill_between(interped_marks, big_16, big_84, color='b', alpha=0.2, label="$16^{th}-84^{th}$")

plt.legend()
plt.show()

In [154]:
interped_marks = np.linspace(0.71, 20, 1000)

small_eta = []
big_eta = []

plt.figure()
plt.title('Eta Over R')
plt.xlabel('R')
plt.ylabel('Eta')
plt.ylim((0,1.1))
plt.xlim((0,20))
for r, f, re, ie in zip(normed_rs, normed_fs, res, ies):
    eta = np.array([f[r==_r]/f[r<=_r].mean() for _r in r]).flatten()
    
    if re<3:
        small_eta.append(np.interp(interped_marks, r, eta))
        plt.plot(r, eta, color='r', alpha=0.2)
    else:
        big_eta.append(np.interp(interped_marks, r, eta))
        plt.plot(r, eta, color='b', alpha=0.2)
        
        
plt.figure()
plt.title('Eta Over R')
plt.xlabel('R')
plt.ylabel('Eta')
stat_vals_big = np.dstack(big_eta)[0,...]
big_mean = np.mean(stat_vals_big, axis=1)
big_16 = np.percentile(stat_vals_big, 16, axis=1)
big_84 = np.percentile(stat_vals_big, 84, axis=1)

stat_vals_small = np.dstack(small_eta)[0,...]
small_mean = np.mean(stat_vals_small, axis=1)
small_16 = np.percentile(stat_vals_small, 16, axis=1)
small_84 = np.percentile(stat_vals_small, 84, axis=1)

plt.plot(interped_marks, small_mean, color='r', label='$mean(R_e < 3)$')
plt.fill_between(interped_marks, small_16, small_84, color='r', alpha=0.2, label="$16^{th}-84^{th}$")

plt.plot(interped_marks, big_mean, color='b', label='$mean(R_e >= 3)$')
plt.fill_between(interped_marks, big_16, big_84, color='b', alpha=0.2, label="$16^{th}-84^{th}$")
plt.legend()
        

In [161]:
small_segmap = []
big_segmap = []

for img, segmap, img_id in _spheroids:
    area = (segmap==img_id).sum()
    
    if img_id in small_ids:
        small_segmap.append(area)
    else:
        big_segmap.append(area)
        
plt.figure()
plt.title('Segmap Area Comparision Small and Big Petrosian Radius Objects')
plt.xlabel('Source Size ($pixels^2$)')
plt.hist(small_segmap, color='r', alpha=0.5, bins=30, label='$R_e < 3$')
plt.hist(big_segmap, color='b', alpha=0.5, bins=100, label='$R_e >= 3$')

(array([ 10.,  18.,  24.,  29.,  25.,  16.,  19.,  15.,  12.,  11.,  17.,
          8.,   6.,   6.,   4.,   6.,   6.,   5.,   6.,   3.,   6.,   1.,
          4.,   3.,   4.,   2.,   3.,   3.,   0.,   4.,   6.,   1.,   2.,
          0.,   1.,   3.,   1.,   1.,   3.,   2.,   0.,   2.,   1.,   1.,
          1.,   1.,   0.,   2.,   2.,   2.,   0.,   1.,   2.,   0.,   1.,
          1.,   0.,   1.,   0.,   0.,   1.,   0.,   0.,   0.,   0.,   0.,
          0.,   2.,   1.,   1.,   0.,   0.,   1.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   1.,   1.,   0.,   0.,   0.,
          0.,   0.,   0.,   2.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          1.]),
 array([  121.  ,   168.69,   216.38,   264.07,   311.76,   359.45,
          407.14,   454.83,   502.52,   550.21,   597.9 ,   645.59,
          693.28,   740.97,   788.66,   836.35,   884.04,   931.73,
          979.42,  1027.11,  1074.8 ,  1122.49,  1170.18,  1217.87,
         1265.56,  1313.25,  1360.94,  1408.63

In [46]:
#def petrosian_radius(rs, fs):
    #r_candidates = np.array([one_over_eta(rs, fs, R) for R in rs])
#    r_candidates = np.array([fs[rs==r]/fs[rs<=r].mean() for r in rs]).flatten()
#    r_candidates = r_candidates[r_candidates<12.5]
    
#    return rs[np.square(r_candidates-0.2).argmin()]


for rs, fs in zip(normed_rs, normed_fs):
    r_candidates = np.array([fs[rs==r]/fs[rs<=r].mean() for r in rs]).flatten()
    r_candidates = 
    
    plt.plot(rs, r_candidates, color='b', alpha=0.2)
    eta = rs[np.square(r_candidates-0.2).argmin()]
    plt.vlines([eta], 0, 1)
    
plt.show()

In [120]:
smalls = []
bigs = []

for img, segmap, img_id in _spheroids:
    if img_id in small_ids:
        smalls.append((img, segmap, img_id))
    else:
        bigs.append((img, segmap, img_id))

In [141]:
sm = smalls[:5]
bg = bigs[:5]

small_coll = []
big_coll = []


for img, segmap, img_id in sm+bg:
    src_map = segmap==img_id
    
    rs, fs = get_rs_and_fs(img, src_map)
    fs = dt.loessc_p(rs, fs, 3.0, pnum=4)
    R_e = get_re((rs,fs))
    
    
    R_p = petrosian_radius(rs, fs)
    p_mask = rs <= 2*R_p

    rs = rs[p_mask]
    fs = fs[p_mask]
    
    R_50 = R_x(rs, fs, 50)
    R_90 = R_x(rs, fs, 90)
    
    
    cx, cy = it.img_center(img, src_map)
    old_re = it.effective_radius(img, src_map)
    
    item = (img, (cx, cy), R_p, R_50, R_90, R_e, old_re)
    
    if R_e < 3:
        small_coll.append(item)
    else:
        big_coll.append(item)


In [144]:
count=1
for (img, (cx, cy), R_p, R_50, R_90, R_e, old_re) in big_coll:
    make_cir = lambda r,c: plt.Circle((cx, cy), r, color=c, fill=False)
    
    cir_p_radius = make_cir(R_p, 'b')
    cir_R_50 = make_cir(R_50, 'y')
    cir_R_90 = make_cir(R_90, 'r')
    cir_R_e = make_cir(R_e, 'g')
    cir_old_re = make_cir(old_re, 'w')

    f, a = plt.subplots(figsize=(15,15))
    #f, a = plt.subplots()
    a.imshow(img, cmap='gray')
    a.add_artist(cir_p_radius)
    a.add_artist(cir_R_50)
    a.add_artist(cir_R_90)
    a.add_artist(cir_R_e)
    a.add_artist(cir_old_re)
    a.legend([cir_p_radius, cir_R_50, cir_R_90, cir_R_e, cir_old_re], 
             ['Petrosian Radius', 'R_50', 'R_90', 'R_e', 'Old Re'],
            fontsize=20)
    plt.savefig(f'big_{count}.png')
    count += 1
    

In [42]:
plt.figure()
plt.title("$R_e$ Found Using Petrosian")
plt.xlabel('$R_e$')
plt.hist(res, bins=100)
plt.show()

In [112]:
small_ids = []

for i, r in small_res:
    small_ids.append(i)

In [37]:
spheroids = get_random_spheroids(num=-1)


In [42]:
s = None
for img, segmap, img_id in spheroids:
    if img_id == 9278:
        s = (img, segmap, img_id)

In [48]:
reload(dt)

img, segmap, img_id = s
img = denoise(img, segmap, img_id)

# print image
plt.figure()
plt.imshow(img, cmap='gray')
plt.show()


rs, fs = get_rs_and_fs(img, segmap==img_id)
fs = dt.loessc_p(rs, fs, 3.0, pnum=1)

plt.figure()
plt.plot(rs, fs)
plt.show()


re = get_re((rs,fs))
print(re)

fs = np.array([fs[rs==r]/fs[rs<=r].mean() for r in rs]).flatten()

plt.figure()
plt.plot(rs, fs, color='b', alpha=0.2)

2.21825353488


In [51]:
plt_rs = []
plt_fs = []

count = 0
for img, segmap, img_id in spheroids:
    count += 1
    print(count/len(spheroids), end='\r')
    #background subtract images
    src_map = segmap==img_id
    
    img = denoise(img, segmap, img_id)
    
    # get the sorted r vals and i vals
    rs, fs = get_rs_and_fs(img, src_map)
    fs = dt.loessc_p(rs, fs, 3.0, pnum=8)
    
    re = get_re((rs,fs))
    re_idx = np.where(rs==re)[0][0] 

    ie = fs[re_idx]
    
    fs = fs[rs<=max_r]/ie
    rs = rs[rs<=max_r]/re
    
    plt_rs.append(rs)
    plt_fs.append(fs)

In [53]:
plt.figure()
for r, f in zip(plt_rs, plt_fs):
    plt.semilogy(r, f, color='b', alpha=0.2)
    


In [ ]:
f_size = (10,10)

plt.figure(figsize=f_size)
plt.title('Cumulative SBP')
plt.ylabel('$\sum_r^R I(r) / \sum_r^{Rmax} I(r)$')
plt.xlabel('$R/R_e$')
for r, f in zip(normed_rs, normed_fs):
    color = (0, 0, 0)
    plt.plot(r, np.cumsum(f)/f.sum(), color=color, alpha=0.25)
plt.plot([1], [0.5], '.')
    
plt.figure(figsize=f_size)
plt.title('$R_e$ Hist')
plt.hist(res, bins=50)

plt.figure(figsize=f_size)
plt.title('$R_e$/Max Hist')
plt.hist(re_max_ratio, bins=50)


plt.show()

In [ ]:
img = _img.copy()
plt.figure()
plt.imshow(np.logical_and(segmap!=0, segmap!=img_id))

src_map = segmap==img_id

noise_bank = img[segmap==0].flatten()
other_source = np.logical_and(segmap!=0, segmap!=img_id)
np.place(img, other_source, noise_bank)

array_sep_likes = img.byteswap().newbyteorder()
bkg = sep.Background(array_sep_likes, mask=segmap==img_id, bw=10, bh=10)
img = img-bkg

print(bkg.globalrms)
plt.figure()
print(img.min(), img.max())
plt.xlim((-0.1, 0.1))
plt.hist(img.flatten(), bins=500)

plt.figure()
plt.imshow(img)
plt.figure()
plt.imshow(bkg)

plt.show()

In [4]:
from importlib import reload
reload(dt)

count = 0
for _img, segmap, img_id in get_random_spheroids(num=100):
    print(count/100, end='\r')
    count += 1
    
    img = denoise(_img, segmap, img_id)

    rs, fs = get_rs_and_fs(img, segmap==img_id)

    _fs = dt.loessc_p(rs, fs, 0.30, pnum=8)
    _fs = np.array([_fs[rs==r]/_fs[rs<=r].mean() for r in rs]).flatten()

    #plt.ylim((0,2))
    plt.plot(rs, _fs, color='b', alpha=0.2)

plt.show()